In [9]:
import numpy as np
import pandas as pd
import pickle
from google.cloud import storage
import category_encoders as ce
import os
from dotenv import load_dotenv
load_dotenv()

In [2]:
bucket_name = 'BUCKET_NAME'
file_name = 'FILE_PATH'
client = storage.Client()

In [3]:
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)

In [4]:
blob.download_to_filename('FILE_PATH')
df = pd.read_csv('FILE_PATH', sep=';')

In [5]:
df

,Date de début,code site,nom site,Polluant,valeur,Latitude,Longitude,type d'implantation,type d'influence
0,2022/08/01,FR01011,Metz-Centre,PM10,0.0,49.119442,6.180833,Urbaine,Fond
1,2022/08/01,FR01012,Metz-Borny,PM10,7.0,49.110283,6.223336,Urbaine,Fond
2,2022/08/01,FR01014,Pont-à-Mousson,PM10,10.0,48.915000,6.058336,Urbaine,Fond
3,2022/08/01,FR01019,Thionville-Piscine,PM10,7.0,49.368610,6.160831,Urbaine,Fond
4,2022/08/01,FR01020,Thionville-Centre,PM10,11.0,49.358337,6.156942,Urbaine,Fond
...,...,...,...,...,...,...,...,...,...
226115,2023/09/30,FR82030,Montandon Baresans,PM10,10.0,47.294640,6.825861,Rurale nationale,Fond
226116,2023/09/30,FR82040,Dole centre,PM10,14.0,47.096720,5.496389,Urbaine,Fond
226117,2023/09/30,FR82043,Chatenois,PM10,12.0,47.147250,5.556117,Périurbaine,Industrielle
226118,2023/09/30,FR82060,Vesoul Pres Caillet,PM10,12.0,47.620022,6.157642,Urbaine,Fond


In [6]:
 df.dtypes

Date de début           object
code site               object
nom site                object
Polluant                object
valeur                 float64
Latitude               float64
Longitude              float64
type d'implantation     object
type d'influence        object
dtype: object

In [7]:
df["type d'implantation"].unique()

array(['Urbaine', 'Périurbaine', 'Rurale régionale',
       'Rurale près des villes', 'Rurale nationale'], dtype=object)

In [8]:
data=df

In [10]:
encoder = ce.OrdinalEncoder(cols=["type d'implantation", "type d'influence"])

In [11]:
data= encoder.fit_transform(data)

In [12]:
data

,Date de début,code site,nom site,Polluant,valeur,Latitude,Longitude,type d'implantation,type d'influence
0,2022/08/01,FR01011,Metz-Centre,PM10,0.0,49.119442,6.180833,1,1
1,2022/08/01,FR01012,Metz-Borny,PM10,7.0,49.110283,6.223336,1,1
2,2022/08/01,FR01014,Pont-à-Mousson,PM10,10.0,48.915000,6.058336,1,1
3,2022/08/01,FR01019,Thionville-Piscine,PM10,7.0,49.368610,6.160831,1,1
4,2022/08/01,FR01020,Thionville-Centre,PM10,11.0,49.358337,6.156942,1,1
...,...,...,...,...,...,...,...,...,...
226115,2023/09/30,FR82030,Montandon Baresans,PM10,10.0,47.294640,6.825861,5,1
226116,2023/09/30,FR82040,Dole centre,PM10,14.0,47.096720,5.496389,1,1
226117,2023/09/30,FR82043,Chatenois,PM10,12.0,47.147250,5.556117,2,2
226118,2023/09/30,FR82060,Vesoul Pres Caillet,PM10,12.0,47.620022,6.157642,1,1


In [13]:
encoder.mapping

[{'col': "type d'implantation",
  'mapping': Urbaine                   1
  Périurbaine               2
  Rurale régionale          3
  Rurale près des villes    4
  Rurale nationale          5
  NaN                      -2
  dtype: int64,
  'data_type': dtype('O')},
 {'col': "type d'influence",
  'mapping': Fond            1
  Industrielle    2
  Trafic          3
  NaN            -2
  dtype: int64,
  'data_type': dtype('O')}]

In [14]:
mapping = encoder.mapping

with open('Dict', 'wb') as f:
    pickle.dump(mapping, f)

In [15]:
data["Date de début"]=pd.to_datetime(data["Date de début"])

In [16]:
data.dtypes

Date de début          datetime64[ns]
code site                      object
nom site                       object
Polluant                       object
valeur                        float64
Latitude                      float64
Longitude                     float64
type d'implantation             int64
type d'influence                int64
dtype: object

In [17]:
object_columns = data.select_dtypes(include=['object']).columns
# Supprimer ces colonnes du DataFrame
data= data.drop(object_columns, axis=1)

In [27]:
data 

,valeur,Latitude,Longitude,type d'implantation,type d'influence,Year,Month,Day
0,0.0,49.119442,6.180833,1,1,2022,8,1
1,7.0,49.110283,6.223336,1,1,2022,8,1
2,10.0,48.915000,6.058336,1,1,2022,8,1
3,7.0,49.368610,6.160831,1,1,2022,8,1
4,11.0,49.358337,6.156942,1,1,2022,8,1
...,...,...,...,...,...,...,...,...
226115,10.0,47.294640,6.825861,5,1,2023,9,30
226116,14.0,47.096720,5.496389,1,1,2023,9,30
226117,12.0,47.147250,5.556117,2,2,2023,9,30
226118,12.0,47.620022,6.157642,1,1,2023,9,30


In [28]:
data['Year'] = data["Date de début"].dt.year
data['Month'] = data["Date de début"].dt.month
data['Day'] = data["Date de début"].dt.day
data=data.drop(['Date de début'], axis=1)

KeyError: 'Date de début'

In [29]:
csv_filename = ''
data.to_csv(csv_filename, index=False)

# Exporter data et Dict 

In [30]:
csv_blob = bucket.blob('FILE_PATH')
csv_blob.upload_from_filename('FILE_PATH')

# Envoyer le fichier pickle sur GCP
pickle_blob = bucket.blob('BUCKET_NAME')
pickle_blob.upload_from_filename('BUCKET_NAME')

In [31]:
data.dtypes

valeur                 float64
Latitude               float64
Longitude              float64
type d'implantation      int64
type d'influence         int64
Year                     int64
Month                    int64
Day                      int64
dtype: object